In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [ ]:
# Set up the endpoint and query parameters.
url = "https://api.linkedin.com/rest/memberSnapshotData"
params = {
    "q": "criteria",
    "domain": "CONNECTIONS"
}

# Set up the headers with your access token and version header.
headers = {
    "Authorization": "Bearer KEY",
    "Content-Type": "application/json",
    "LinkedIn-Version": "202312"
}

# Make the GET request to the Member Snapshot API.
response = requests.get(url, params=params, headers=headers)

# Process the response.
if response.status_code == 200:
    snapshot_data = response.json()

    # Assuming snapshot_data contains an 'elements' list with one element for the CONTACTS domain.
    if 'elements' in snapshot_data and len(snapshot_data['elements']) > 0:
        contacts_snapshot = snapshot_data['elements'][0].get('snapshotData', [])

        # Convert the contacts list into a Pandas DataFrame.
        df = pd.DataFrame(contacts_snapshot)
    else:
        print("No snapshot data found for the CONTACTS domain.")
else:
    print("Error fetching snapshot data:", response.status_code)
    print(response.text)

df


In [ ]:
def fetch_member_snapshot(api_key, domain="CONNECTIONS"):
    # Set up the endpoint and query parameters.
    url = "https://api.linkedin.com/rest/memberSnapshotData"
    params = {
        "q": "criteria",      # Required query parameter
        "domain": domain      # Querying the specified domain (e.g. CONNECTIONS)
    }

    # Set up the headers with the provided API key and version header.
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "LinkedIn-Version": "202312"  # Ensure you use the correct API version
    }

    # Make the GET request to the Member Snapshot API.
    response = requests.get(url, params=params, headers=headers)

    # Process the response.
    if response.status_code == 200:
        snapshot_data = response.json()

        # Assuming snapshot_data contains an 'elements' list with one element.
        if 'elements' in snapshot_data and len(snapshot_data['elements']) > 0:
            snapshot = snapshot_data['elements'][0].get('snapshotData', [])
            # Convert the snapshot list into a Pandas DataFrame.
            return pd.DataFrame(snapshot)
        else:
            print("No snapshot data found for the domain:", domain)
            return pd.DataFrame()
    else:
        print("Error fetching snapshot data:", response.status_code)
        print(response.text)
        return pd.DataFrame()

# Store different API keys in variables.
jan_key = "KEY"

lars_key = "KEY"

pascal_key = "KEY"

ina_key = "KEY"
# You can add more keys as needed.

# Make the same API call for different API keys and store results in separate DataFrames.
jan_df = fetch_member_snapshot(jan_key, domain="CONNECTIONS")
lars_df = fetch_member_snapshot(lars_key, domain="CONNECTIONS")
pascal_df = fetch_member_snapshot(pascal_key, domain="CONNECTIONS")
ina_df = fetch_member_snapshot(ina_key, domain="CONNECTIONS")

In [ ]:
jan_df

In [ ]:
lars_df

In [ ]:
pascal_df

In [ ]:
ina_df

In [ ]:
def process_connections_data(df, label="DataFrame"):
    # Convert 'Connected On' to datetime using the provided format.
    df['Connected On'] = pd.to_datetime(df['Connected On'], format='%d %b %Y')

    # Extract the year from 'Connected On' and count connections per year.
    connections_per_year = df['Connected On'].dt.year.value_counts().sort_index()

    # Ensure that all years in the range are represented.
    all_years = range(df['Connected On'].dt.year.min(), df['Connected On'].dt.year.max() + 1)
    connections_per_year = connections_per_year.reindex(all_years, fill_value=0)

    # Plot the data: line with dot markers.
    plt.figure(figsize=(8, 5))
    plt.plot(connections_per_year.index, connections_per_year.values, marker='o', linestyle='-', linewidth=2)
    plt.xticks(all_years)  # Ensure the x-axis displays all years.
    plt.xlabel('Year')
    plt.ylabel('Number of Connections')
    plt.title(f'New LinkedIn Connections per Year - {label}')
    plt.grid(True)
    plt.show()


# Organize the DataFrames into a dictionary for easy iteration.
dataframes = {
    "Jan": jan_df,
    "Lars": lars_df,
    "Pascal": pascal_df,
    "Ina": ina_df,
}

# Loop over each DataFrame, processing and plotting the results.
for label, df in dataframes.items():
    process_connections_data(df, label=label)


In [ ]:
jan_total = len(jan_df)
lars_total = len(lars_df)
pascal_total = len(pascal_df)
ina_total = len(ina_df)

datasets = ['Jan', 'Lars', 'Pascal', 'Ina']
total_connections = [jan_total, lars_total, pascal_total, ina_total]

plt.figure(figsize=(6, 4))
plt.bar(datasets, total_connections, color=['blue', 'orange', 'green', "red"])
plt.xlabel('Dataset')
plt.ylabel('Total LinkedIn Connections')
plt.title('Total LinkedIn Connections - Comparison')
plt.show()